In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
data = []

with open('/content/drive/MyDrive/Y_OP/Test1/VID_20201110_170806-yolo.json') as json_file:
  data = json.load(json_file)

In [ ]:
import pandas as pd

df_rows = list()
for frame_id, frame in enumerate(data):
    for obj in frame:
        df_rows.append([frame_id, obj['name'], obj['percentage_probability'], *obj['box_points']])

df = pd.DataFrame(df_rows, columns=["frame", "object", "proba", "x1", "y1", "x2", "y2"])
print(df)

all_frames = df["frame"].max() # ilość wszystkich klatek
print(all_frames)
all= df['object'].value_counts()
print(all) # drukuje wszystkie

# stworzenie dataframe z rim, gdzie jest ponad 90 % 
rim = df[(df['object']) == 'rim']
gproba_rim = rim[rim['proba'] > 90]
 # stworzenie dataframe z ball, gdzie jest ponad 60 % 
ball = df[(df['object']) == 'ball']
gproba_ball = ball[ball['proba'] > 60]
# stworzenie dataframe z people, gdzie jest ponad 80 % 
people = df[(df['object']) =='people']
gproba_pp= people[people['proba'] > 80]
# połączenie trzech dataframe w jedno
df=pd.concat([gproba_pp, gproba_ball,gproba_rim])

df_yolo_all=pd.concat([gproba_pp, gproba_ball,gproba_rim])

all_good = df['object'].value_counts()
print(all_good) # drukuje prawidłowe

     frame  object      proba    x1   y1    x2   y2
0        0     rim  99.824744  1344  238  1484  344
1        1     rim  99.820614  1343  238  1485  344
2        1    ball  50.594467   280  704   335  737
3        2     rim  99.814391  1344  238  1485  344
4        3     rim  99.811375  1344  238  1485  344
..     ...     ...        ...   ...  ...   ...  ...
232     98     rim  99.751657  1341  235  1485  347
233     98    ball  94.189703   953  293  1009  343
234     99     rim  99.758506  1340  236  1487  346
235     99    ball  89.972377   936  317   994  369
236     99  people  61.721522   249  507   379  882

[237 rows x 7 columns]
99
rim       100
ball       75
people     62
Name: object, dtype: int64
rim       100
ball       67
people     35
Name: object, dtype: int64


In [ ]:
# sprawdzanie ile klatek jest poprawnych (3 elementy)
count_frames = df["frame"].max()
n = 0
is_good = 0 # zmienna przechwująca dobre klatki
bad_frames = [] # lista przechowująca złe klatki (potrzebne do openpose)
good_frames = [] # lista przechowująca dobre klatki
while n < count_frames: 
  one_frame = df[(df['frame']) == n]
  x = one_frame['object'].count() # zliczanie ile mamy obiektów w  danej klatce
  if "rim" in one_frame.values and "people" in one_frame.values and 'ball' in one_frame.values and x == 3:
    print("Klatka",n,"jest dobra")
    is_good += 1
    good_frames.append(n)
  else:
    print("Klatka",n,"nie jest dobra")
    df = df[df.frame != n]
    bad_frames.append(n)
  n += 1
print("Dobrych klatek:", is_good)
ratio = is_good/count_frames  # jaki współczynnik klatek jest dobry
print("Wspolczynnik dobrych:", ratio)
if ratio > 0.5 :
  print("Film się nadaje ")
else:
  print("Film się nie nadaje")

Klatka 0 nie jest dobra
Klatka 1 nie jest dobra
Klatka 2 nie jest dobra
Klatka 3 nie jest dobra
Klatka 4 nie jest dobra
Klatka 5 nie jest dobra
Klatka 6 nie jest dobra
Klatka 7 nie jest dobra
Klatka 8 nie jest dobra
Klatka 9 nie jest dobra
Klatka 10 nie jest dobra
Klatka 11 nie jest dobra
Klatka 12 nie jest dobra
Klatka 13 nie jest dobra
Klatka 14 nie jest dobra
Klatka 15 nie jest dobra
Klatka 16 nie jest dobra
Klatka 17 nie jest dobra
Klatka 18 nie jest dobra
Klatka 19 nie jest dobra
Klatka 20 nie jest dobra
Klatka 21 nie jest dobra
Klatka 22 nie jest dobra
Klatka 23 nie jest dobra
Klatka 24 nie jest dobra
Klatka 25 nie jest dobra
Klatka 26 nie jest dobra
Klatka 27 nie jest dobra
Klatka 28 nie jest dobra
Klatka 29 nie jest dobra
Klatka 30 nie jest dobra
Klatka 31 nie jest dobra
Klatka 32 nie jest dobra
Klatka 33 nie jest dobra
Klatka 34 nie jest dobra
Klatka 35 nie jest dobra
Klatka 36 nie jest dobra
Klatka 37 nie jest dobra
Klatka 38 nie jest dobra
Klatka 39 nie jest dobra
Klatka 40 

In [ ]:
# i teraz trzeba sprawdzić na ilu klatkach widać przechodzącą pilke przez obrecz (trafiony)
for n in good_frames:
  one_frame= df[(df['frame']) == n]
# wyciągamy koordynaty piłki
  ball_cords = one_frame[(one_frame['object']) == 'ball']
  b_x1 = ball_cords['x1'].item()
  b_x2 = ball_cords['x2'].item()
  b_y1 = ball_cords['y1'].item()
  b_y2 = ball_cords['y2'].item()
# wyliczamy środek piłki
  b_x = (b_x1 + b_x2)/2
  b_y = (b_y1 + b_y2)/2
# wyciągamy koordynaty ramki
  rim_cords = one_frame[(one_frame['object']) == 'rim']
  r_x1 = rim_cords['x1'].item()
  r_x2 = rim_cords['x2'].item()
  r_y1 = rim_cords['y1'].item()
  r_y2 = rim_cords['y2'].item()
# sprawdzamy czy środek piłki przeszedł przez ramkę
  if b_x1 == r_x1 and b_x2 == r_x2 and b_y1 == r_y1 and b_y2 == r_y2:
    df = df[df.frame != n]
  elif r_x1 < b_x and r_x2 > b_x and r_y1 < b_y and r_y2 > b_y:
    print(n, "TRAFIONE!")
  else:
    print(n, "nie trafione :(")

44 nie trafione :(
45 nie trafione :(
49 nie trafione :(
50 nie trafione :(
56 nie trafione :(
57 nie trafione :(
58 nie trafione :(
59 nie trafione :(
60 nie trafione :(
61 nie trafione :(
62 nie trafione :(
63 nie trafione :(
64 nie trafione :(
65 nie trafione :(
66 nie trafione :(
67 nie trafione :(
76 nie trafione :(
89 nie trafione :(
90 nie trafione :(
91 nie trafione :(
94 nie trafione :(
96 nie trafione :(


In [ ]:
#pobranie dataframe openpose
with open('/content/drive/MyDrive/Y_OP/Test1/VID_20201110_170806-openpose.json') as json_file:
  data_op = json.load(json_file)

df_rows = list()
for frame_id, frame in enumerate(data_op):
  for person_id, person in enumerate(frame['persons']):
    for part_id, part in enumerate(person):
      df_rows.append([frame_id, person_id, part_id, *part])

df_op = pd.DataFrame(df_rows, columns=["frame", "person", "part", "x", "y", "proba"])
df_op_all = pd.DataFrame(df_rows, columns=["frame", "person", "part", "x", "y", "proba"])
# usuwanie klatek w których YOLO nie wykryło 3 dobrych obiektów 
#   (tymczasowo usunięte bo chyba lepiej wykrywać czy punkt z op znajduje się w kwadracie z YOLO na każdej klatce)
# for i in bad_frames:
#   df_op = df_op[df_op.frame != i]
# print(df_op)

# zobaczenie czy środek ciała w ogóle jest na klatce xD
# test = df_op[df_op['part'] == 8]
# print(test)

# usuwanie niepotrzebych punktów
points = [15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
for i in points:
  df_op = df_op[df_op.part != i]
# print(df_op)

# sprawdzenie wiekszego prawdopodobieństwa i usunięcie osób jeśli jest więcej niż jedna
for frame_id, df_frame in df_op.groupby("frame"):
  probas = []
  for person_id, df_person in df_frame.groupby("person"):
    probas.append(df_person['proba'].mean())
  person_correct_id = probas.index(max(probas))
  df_op = df_op[(df_op["frame"] != frame_id) | ((df_op["frame"] == frame_id) & (df_op["person"] == person_correct_id))]
  df_op_all = df_op[(df_op["frame"] != frame_id) | ((df_op["frame"] == frame_id) & (df_op["person"] == person_correct_id))]
# jaka osoba została na klatce
#for frame_id, df_frame in df_op.groupby("frame"):
#   print(df_frame["person"].unique())

print(df_op)

      frame  person  part        x        y     proba
0         0       0     0  288.889  597.848  0.917221
1         0       0     1  230.078  627.393  0.781033
2         0       0     2  233.060  633.253  0.829421
3         0       0     3  244.721  709.805  0.898068
4         0       0     4  294.804  739.328  0.938137
5         0       0     5  227.132  624.419  0.780477
6         0       0     6  247.785  698.042  0.695508
7         0       0     7  294.719  715.740  0.869334
8         0       0     8  230.130  765.662  0.821047
9         0       0     9  244.657  765.734  0.842042
10        0       0    10  268.407  874.638  0.925374
11        0       0    11  235.975  998.167  0.814440
12        0       0    12  221.361  765.634  0.822017
13        0       0    13  230.089  880.458  0.906424
14        0       0    14  221.077  992.374  0.837008
50        1       0     0  291.815  597.828  0.929278
51        1       0     1  235.915  624.628  0.753246
52        1       0     2  2

In [ ]:
# Sprawdzamy czy człowiek jest w człowieku xd
one_frame_yolo = df[(df['frame'].isin(good_frames)) & (df['object'] == 'people')]
one_frame_op = df_op[(df_op['frame'].isin(good_frames)) & (df_op['part'] == 8)]
# merge obu dataframe
person_check = pd.merge(one_frame_yolo, one_frame_op, how='inner', on='frame')
# sprawdzamy czy środek człowieka z OpenPose znajduje się wewnątrz ramki People z Yolo
def person_condition(x1, x2, x, y1, y2, y):
  if x1 < x and x2 > x and y1 < y and y2 > y:
    return True
  return False
  
person_check['good'] = [person_condition(*row) for row in zip(person_check['x1'], person_check['x2'], person_check['x'], person_check['y1'], person_check['y2'], person_check['y'])]
person_ratio = person_check['good'].value_counts().loc[True] / person_check.shape[0]

if person_ratio > 0.90:
  print("W pyte film", person_ratio)
else:
  print("Dej nowy", person_ratio)

W pyte film 1.0


In [ ]:
yolo_ball_frames = []  # lista dobrych ramek
for frame_id, df_frame in df_yolo_all.groupby("frame"):
  ball_counter = df_frame['object'].value_counts()
  if 'ball' in ball_counter.index and ball_counter.loc['ball'] == 1:
    yolo_ball_frames.append(frame_id)

df_yolo_all=df_yolo_all[(df_yolo_all["object"] == 'ball') & (df_yolo_all['frame'].isin(yolo_ball_frames))]

# środek piłki
df_yolo_all['x_center'] = (df_yolo_all['x1'] + df_yolo_all['x2'])/2
df_yolo_all['y_center'] = (df_yolo_all['y1'] + df_yolo_all['y2'])/2

df_op_all = df_op_all[(df_op_all['frame'].isin(yolo_ball_frames)) & (df_op_all['part'].isin([1, 4, 7, 8]))] # klatki 

dict_x_y = {} # słownik częsci ciała
def part_x_y(d, frame_id, part_id, p_x, p_y): # d - słownik
  frame_key = 'frame_' + str(frame_id) # klucz słownika
  part_key = '_part_' + str(part_id) # nazwa części ciała - klucz
  if frame_key not in d.keys(): 
    d[frame_key] = {'frame': frame_id} # jeśli ramki nie ma to musi ją utworzyć w słowniku 
  d[frame_key]['x' + part_key] = p_x # współrzędna x
  d[frame_key]['y' + part_key] = p_y # współrzędna y
  
_ = [part_x_y(dict_x_y, *row) for row in zip(df_op_all['frame'], df_op_all['part'], df_op_all['x'], df_op_all['y'])] # wyrażenie listowe

df_concat = pd.merge(df_yolo_all, pd.DataFrame(list(dict_x_y.values())), how='inner', on='frame')
df_concat.drop(['object', 'proba', 'x1', 'y1', 'x2', 'y2'], axis=1, inplace=True)
df_concat

,frame,x_center,y_center,x_part_1,y_part_1,x_part_4,y_part_4,x_part_7,y_part_7,x_part_8,y_part_8
0,4,385.0,938.0,244.896,627.439,315.449,736.226,300.778,718.706,244.643,765.708
1,5,384.0,901.0,244.895,627.459,321.316,739.156,318.490,733.334,244.671,765.755
2,7,383.0,841.0,247.827,627.510,342.004,739.268,338.955,739.363,244.780,765.863
3,10,385.0,772.5,250.691,624.555,362.515,739.122,327.232,739.209,247.656,765.791
4,17,376.5,715.5,271.183,621.614,344.824,715.771,341.840,715.823,259.496,765.655
5,18,376.0,717.0,271.200,621.641,341.834,715.642,333.092,718.531,262.456,762.880
6,19,355.5,715.0,271.328,624.365,341.835,715.592,336.013,718.581,262.472,765.660
7,24,353.0,713.0,274.329,621.529,336.068,698.121,330.096,718.547,262.400,762.776
8,25,355.0,711.5,277.138,621.442,336.037,698.045,327.195,715.590,262.438,762.784
9,26,354.5,710.0,283.021,621.381,335.976,695.256,333.123,709.794,256.619,762.752


In [ ]:
import math

tab_1 = []
tab_4 = []
tab_7 = []
tab_8 = []

for n in yolo_ball_frames:
  one_frame = df_concat[(df_concat['frame']) == n]

# wyciągamy koordynaty punktów i środka piłki
  x_1 = one_frame['x_part_1'].item()
  y_1  = one_frame['y_part_1'].item()
  x_center = one_frame['x_center'].item()
  y_center  = one_frame['y_center'].item()

  x_4 = one_frame['x_part_4'].item()
  y_4  = one_frame['y_part_4'].item()

  x_7 = one_frame['x_part_7'].item()
  y_7  = one_frame['y_part_7'].item()

  x_8 = one_frame['x_part_8'].item()
  y_8  = one_frame['y_part_8'].item()

# obliczamy odległość między środkiem piłki a punktami 

  d_1 = math.sqrt((x_center - x_1)**2 + (y_center - y_1)**2 )
  d_4 = math.sqrt((x_center - x_4)**2 + (y_center - y_4)**2 )
  d_7 = math.sqrt((x_center - x_7)**2 + (y_center - y_7)**2 )
  d_8 = math.sqrt((x_center - x_8)**2 + (y_center - y_8)**2 )

  tab_1.append(d_1)
  tab_4.append(d_4)
  tab_7.append(d_7)
  tab_8.append(d_8)

df_concat['dist_1'] = tab_1
df_concat['dist_4'] = tab_4
df_concat['dist_7'] = tab_7
df_concat['dist_8'] = tab_8

# pd.set_option('display.max_rows', None)  # pokazuje cały datafrane
# pd.set_option('display.max_columns', None)
df_concat.head()



,frame,x_center,y_center,x_part_1,y_part_1,x_part_4,y_part_4,x_part_7,y_part_7,x_part_8,y_part_8,dist_1,dist_4,dist_7,dist_8
0,4,385.0,938.0,244.896,627.439,315.449,736.226,300.778,718.706,244.643,765.708,340.701138,213.424667,234.911055,222.226508
1,5,384.0,901.0,244.895,627.459,321.316,739.156,318.490,733.334,244.671,765.755,306.879259,173.559109,180.009577,194.174613
2,7,383.0,841.0,247.827,627.510,342.004,739.268,338.955,739.363,244.780,765.863,252.685021,109.681684,110.770221,157.322399
3,10,385.0,772.5,250.691,624.555,362.515,739.122,327.232,739.209,247.656,765.791,199.816492,40.245076,66.674077,137.507763
4,17,376.5,715.5,271.183,621.614,344.824,715.771,341.840,715.823,259.496,765.655,141.089516,31.677159,34.661505,127.300668


In [ ]:
# 1 pozycja - piłka najbliżej punktu  8
min_8 = df_concat['dist_8'].min()
four_frames= []
for n in yolo_ball_frames:
  one_frame = df_concat[(df_concat['frame']) == n]
  eight = one_frame['dist_8'].item()
  if eight == min_8:
    four_frames.append(n)

df_concat_clean = df_concat[df_concat['frame'] >= four_frames[0]] # usunięcie wszystkich klatek przed 1 pozycją

# 2 pozycja

# 3 i 4 pozycja - piłka ostatni raz dotyka rąk (punkt 4 i 7) i pierwszy raz ich nie dotyka
epsilon = 50
df_concat_clean = df_concat_clean[['dist_4','dist_7']].diff().apply(abs) # różnica między poprzednimi ramkami 
df_concat_clean['sub'] = (df_concat_clean['dist_4'] - df_concat_clean['dist_7']).apply(abs)
df_concat_clean = df_concat_clean[df_concat_clean['sub'] < 50] # usuwamy ramki w których różnica odległości rąk od środka piłki jest za duża (granica błędu)

# srednia ruchoma da 5 klatek, jeśli jest za duża to rzut jest w tych klatkach
df_clean_average = df_concat_clean.rolling(5, center=True).sum()
df_clean_average = df_clean_average[(df_clean_average['dist_4'] >= 100) | (df_clean_average['dist_7'] >=100)]

four_frames.append(df_concat.loc[df_clean_average.index[0], 'frame']) # 3 pozycja
four_frames.append(df_concat.loc[df_clean_average.index[0] + 1, 'frame']) # 4 pozycja

print(four_frames)

[24, 51, 53]
